<h4>Web-scraping Auckland CBD Business Directory</h4>

<h5>Import Dependencies</h5>

In [ ]:
!pip install w3lib
!pip install selenium
!pip install geocoder
!pip install geopy
!pip install geopy
!pip install folium

import numpy as np  # library to handle data in a vectorized manner
import pandas as pd  # library for data analsysis
import requests
import csv
import pprint as pp
import time
import re
import urllib.parse

from w3lib.url import url_query_parameter
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

# from selenium.common.exceptions import IndexError

from bs4 import BeautifulSoup as bs  # Beautiful Soup v4

# Only use when full dataframe needs to be viewed
# pd.set_option("display.max_rows", None, "display.max_columns", None)

import geocoder  # import geocoder

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium  # plotting library

print('Libraries imported.')

<h5>Define Directory Start Page</h5>

In [ ]:
url = 'DIRECTORY START PAGE'  # Typically where the base information about the directory resides
html = requests.get(url).text
soup = bs(html, 'html.parser')

<h5>Set Up Web Control</h5>

In [ ]:
options = Options()
options.page_load_strategy = 'eager'
# Add options you desire, e.g. headless

driver = wd.Chrome(
    'FULL DRIVER EXECUTABLE PATH\chromedriver.exe', options=options)

# View target url to test functionality of driver (won't work in headless mode)
driver.get(url)

<h5>Get Total Count of Directory Items</h5>

In [ ]:
count = soup.find(TARGET THE DIRECTORY COUNT IN RELEVANT ELEMENT)
count = count.string      # convert it to string,
k = int(count)            # then integer,
print(k)                  # and print it to confirm

<h5>Create Lists for Data Storage</h5>

In [ ]:
name_l = []
main_category_l = []
sub_category1_l = []
sub_category2_l = []
str_addy_pre_l = []
str_addy_l = []
precinct_l = []
postcode_l = []
locality_l = []
phone_l = []
website_l = []

<h5>Set Up and Run Data Extraction Loop</h5>

In [ ]:
b = 0             # Item index per page
d = 0             # Page index
e = b + (d * ITEMS PER PAGE)  # Total items accessed

for e in range(0, k):
    try:
        container = soup.find_all(GROUP ELEMENT CONTAINING DIRECTORY INFO)[b]
        
        name = container.splitlines()[LINE NUMBER CONTAINING NAME]           # Always required to exist in the directory
        try:
            main_category = container.splitlines()[LINE NUMBER CONTAINING MAIN CATEGORY]
        except:
            main_category = 'Uncategorized'
        try:
            sub_category1 = container.splitlines()[LINE NUMBER CONTAINING SUB CATEGORY 1]
        except:
            sub_category1 = ''
        try:
            sub_category2 = container.splitlines()[LINE NUMBER CONTAINING SUB CATEGORY 2]
        except:
            sub_category2 = ''
        try:
            str_addy_pre = container.splitlines()[LINE NUMBER CONTAINING ADDRESS FIELD 1]
        except:
            str_addy_pre = ''
        try:
            str_addy = container.splitlines()[LINE NUMBER CONTAINING ADDRESS FIELD 2]
        except:
            str_addy = ''
            
        precinct = container.splitlines()[LINE NUMBER CONTAINING PRECINCT]   # Always added by directory administrator
        postcode = container.splitlines()[LINE NUMBER CONTAINING POSTCODE]   # Always added by directory administrator
        locality = container.splitlines()[LINE NUMBER CONTAINING PRECINCT]   # Always added by directory administrator

        try:
            phone = container.splitlines()[LINE NUMBER CONTAINING PHONE NUMBER]
        except:
            phone = ''        
        try:
            website = container.splitlines()[LINE NUMBER CONTAINING WEBSITE URL]
        except:
            website = ''

    except IndexError:                    # An index error indicates end of page
        b = 0                             # Reset item index per page
        
        # Use Selenium to control current webpage by accessing the next page
        clicker = driver.FIND ELEMENT TO ACCESS NEXT PAGE
        clicker.click()                   # Execute click    
        current_url = driver.current_url  # get the current url
        print(current_url)                # print the current url to confirm
        d = SERIAL PAGE NUMBER FROM CURRENT URL
        d = int(d)

        html = requests.get(current_url).text
        soup = bs(html, 'html.parser')

        container = soup.find_all(GROUP ELEMENT CONTAINING DIRECTORY INFO)[b]
        
        name = container.splitlines()[LINE NUMBER CONTAINING NAME]           # Always required to exist in the directory
        try:
            main_category = container.splitlines()[LINE NUMBER CONTAINING MAIN CATEGORY]
        except:
            main_category = 'Uncategorized'
        try:
            sub_category1 = container.splitlines()[LINE NUMBER CONTAINING SUB CATEGORY 1]
        except:
            sub_category1 = ''
        try:
            sub_category2 = container.splitlines()[LINE NUMBER CONTAINING SUB CATEGORY 2]
        except:
            sub_category2 = ''
        try:
            str_addy_pre = container.splitlines()[LINE NUMBER CONTAINING ADDRESS FIELD 1]
        except:
            str_addy_pre = ''
        try:
            str_addy = container.splitlines()[LINE NUMBER CONTAINING ADDRESS FIELD 2]
        except:
            str_addy = ''
            
        precinct = container.splitlines()[LINE NUMBER CONTAINING PRECINCT]   # Always added by directory administrator
        postcode = container.splitlines()[LINE NUMBER CONTAINING POSTCODE]   # Always added by directory administrator
        locality = container.splitlines()[LINE NUMBER CONTAINING PRECINCT]   # Always added by directory administrator

        try:
            phone = container.splitlines()[LINE NUMBER CONTAINING PHONE NUMBER]
        except:
            phone = ''        
        try:
            website = container.splitlines()[LINE NUMBER CONTAINING WEBSITE URL]
        except:
            website = ''

    name_l.append(name)
    main_category_l.append(main_category)
    sub_category1_l.append(sub_category1)
    sub_category2_l.append(sub_category2)
    str_addy_pre_l.append(str_addy_pre)
    str_addy_l.append(str_addy)
    precinct_l.append(precinct)
    postcode_l.append(postcode)
    locality_l.append(locality)
    phone_l.append(phone)
    website_l.append(website)
    
    # Increment indices
    b += 1
    e = b + (d * ITEMS PER PAGE)  
    print(e)     # To keep track of count

driver.quit()
print("Web Scrapping done.......")

<h5>Create, Populate and View DataFrame</h5>

In [ ]:
data = {'Name': name_l,
 'Main Category': main_category_l,
 'Sub Category 1': sub_category1_l,
 'Sub Category 2': sub_category2_l,
 'Address Line 1': str_addy_pre_l,
 'Address Line 2': str_addy_l,
 'Precinct': precinct_l,
 'Post Code': postcode_l,
 'Locality': locality_l,
 'Phone': phone_l,
 'Web URL': website_l
}

akl_cbd_biz_extract = pd.DataFrame(data)
akl_cbd_biz_extract.head()

<h5>Save DataFrame as Excel File</h5>

In [ ]:
akl_cbd_biz_extract.to_excel(
    r'FULL FILE PATH\FILE.xlsx', index=False, header=True)  # Saved as "web_scraper.xlsx"

print("Saved to excel file")